<a href="https://colab.research.google.com/github/adysinghh/T--T-GAN/blob/main/T_gan_good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from PIL import Image
import os

# Function to generate a single 32x32 RGB image with the letter "T" and a random horizontal line
def generate_image(horizontal_line_position):
    # Initialize a 32x32 image with white background
    image = np.ones((32, 32, 3), dtype=np.uint8) * 255

    # Define the color for the "T" shape (black)
    color = np.zeros((3,), dtype=np.uint8)  # Black color

    # Draw the vertical line of "T"
    image[:, 12:14] = color  # Vertical line of "T" (2 pixels wide)

    # Draw the horizontal line of "T" at the given position with increased thickness
    if horizontal_line_position >= 0:
        image[horizontal_line_position:horizontal_line_position + 2, 6:26] = color  # Thicker horizontal line of "T" (20 pixels wide, 2 pixels thick)

    return image

# Function to create a dataset
def create_dataset(num_images, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i in range(num_images):
        # Randomly choose the horizontal line position within image bounds
        horizontal_line_position = np.random.randint(0, 32)  # Random position from 0 to 31
        image = generate_image(horizontal_line_position)
        img = Image.fromarray(image, 'RGB')
        img.save(os.path.join(save_dir, f'image_{i}.png'))

# Number of images to generate
num_images = 1000  # Adjust as needed
save_dir = 'T_dataset'

# Generate and save the dataset
create_dataset(num_images, save_dir)

print(f'Dataset generated and saved in {save_dir}')

# PyTorch part
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os

# Define the custom dataset
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        if self.transform:
            image = self.transform(image)
        return image, 0  # We don't need labels for GANs, so we return a dummy label

class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1)

# Hyperparameters
nz = 120  # Size of the latent vector (input to the generator)
lr = 0.0001
batch_size = 128
num_epochs = 1200

# Data preprocessing and loading
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((32, 32)),  # Resize images to 32x32
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = CustomImageDataset(image_dir='T_dataset', transform=transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
netG = Generator(nz).to(device)
netD = Discriminator().to(device)

# Loss function and optimizers
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

# Training loop
for epoch in range(num_epochs):
    for i, (data, _) in enumerate(dataloader):
        # Update discriminator
        netD.zero_grad()
        real_data = data.to(device)
        batch_size = real_data.size(0)
        labels = torch.full((batch_size,), 1, dtype=torch.float, device=device)

        output = netD(real_data)
        errD_real = criterion(output, labels)
        errD_real.backward()

        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake_data = netG(noise)
        labels.fill_(0)

        output = netD(fake_data.detach())
        errD_fake = criterion(output, labels)
        errD_fake.backward()
        optimizerD.step()

        # Update generator
        netG.zero_grad()
        labels.fill_(1)

        output = netD(fake_data)
        errG = criterion(output, labels)
        errG.backward()
        optimizerG.step()

        # Print training stats
        if i % 50 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(dataloader)}] "
                  f"Loss D: {errD_real.item() + errD_fake.item():.4f}, Loss G: {errG.item():.4f}")

# Save the models
torch.save(netG.state_dict(), 'generator.pth')
torch.save(netD.state_dict(), 'discriminator.pth')

print("Training completed and models saved.")


Dataset generated and saved in T_dataset
Epoch [1/1200] Batch [0/8] Loss D: 1.5950, Loss G: 0.7502
Epoch [2/1200] Batch [0/8] Loss D: 0.3322, Loss G: 2.0094
Epoch [3/1200] Batch [0/8] Loss D: 0.1804, Loss G: 2.6188
Epoch [4/1200] Batch [0/8] Loss D: 0.1029, Loss G: 3.1705
Epoch [5/1200] Batch [0/8] Loss D: 0.0782, Loss G: 3.3777
Epoch [6/1200] Batch [0/8] Loss D: 0.0759, Loss G: 3.6046
Epoch [7/1200] Batch [0/8] Loss D: 0.0750, Loss G: 3.6795
Epoch [8/1200] Batch [0/8] Loss D: 0.0912, Loss G: 3.7297
Epoch [9/1200] Batch [0/8] Loss D: 1.9622, Loss G: 6.4575
Epoch [10/1200] Batch [0/8] Loss D: 0.7902, Loss G: 1.9705
Epoch [11/1200] Batch [0/8] Loss D: 0.4366, Loss G: 2.0440
Epoch [12/1200] Batch [0/8] Loss D: 0.2358, Loss G: 2.9733
Epoch [13/1200] Batch [0/8] Loss D: 0.1594, Loss G: 3.1679
Epoch [14/1200] Batch [0/8] Loss D: 0.0855, Loss G: 3.7800
Epoch [15/1200] Batch [0/8] Loss D: 0.0697, Loss G: 4.1705
Epoch [16/1200] Batch [0/8] Loss D: 0.0312, Loss G: 4.5057
Epoch [17/1200] Batch [0

In [3]:
import torch
import torch.nn as nn
from torchvision.utils import save_image
import os
from PIL import Image
import torchvision.transforms as transforms

# Define the Generator class
class Generator(nn.Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, 256, 4, 1, 0, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 1, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
nz = 120  # Size of the latent vector (input to the generator)
num_images = 100  # Number of images to generate
image_size = 128  # Size of the output image

# Initialize the generator
netG = Generator(nz).to(device)

# Load the saved model
netG.load_state_dict(torch.load('generator.pth', map_location=device))
netG.eval()  # Set to evaluation mode

# Create output directory if it doesn't exist
output_dir = 'high_quality_generated_images'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Define image transformation for upscaling
upscale_transform = transforms.Compose([
    transforms.Resize(image_size, interpolation=Image.BICUBIC),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1))  # Convert grayscale to RGB
])

# Generate images
with torch.no_grad():
    for i in range(num_images):
        # Generate random noise
        noise = torch.randn(1, nz, 1, 1, device=device)

        # Generate fake image
        fake_image = netG(noise)

        # Rescale images to be between 0 and 1
        fake_image = (fake_image + 1) / 2.0

        # Upscale and convert to RGB
        fake_image = upscale_transform(fake_image.squeeze(0))

        # Save the image
        save_image(fake_image, f'{output_dir}/high_quality_generated_image_{i+1}.png',
                   nrow=1, padding=0, normalize=False)

print(f"Generated {num_images} high-quality images in the '{output_dir}' directory.")


Generated 100 high-quality images in the 'high_quality_generated_images' directory.
